In [1]:
!pip install -q sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 9.8 MB/s eta 0:00:00


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [3]:
#%load_ext cuml.accel
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

2025-08-21 09:54:33.012845: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755770073.176032      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755770073.222081      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
df= pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [5]:
df.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [6]:
X=df.drop(columns=["id","target"])
y=df["target"]

In [7]:
class SBERTTransformer(BaseEstimator, TransformerMixin):
    _shared_model = None

    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2", show_progress_bar=False):
        self.model_name = model_name
        self.show_progress_bar = show_progress_bar

        if SBERTTransformer._shared_model is None:
            print(f"Loading SBERT model: {self.model_name}")
            SBERTTransformer._shared_model = SentenceTransformer(self.model_name)

        self.model = SBERTTransformer._shared_model

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Ensure X is 1D array of strings
        if isinstance(X, pd.DataFrame):
            X = X.squeeze()
        X_filled = np.where(pd.isna(X), "Nan", X)
        return np.array(
            self.model.encode(X_filled.tolist(), show_progress_bar=self.show_progress_bar)
        )

In [8]:
preprocessor = ColumnTransformer(
    transformers=[
        ('text_embed', SBERTTransformer(), 'text')
    ],
    remainder='drop'
)

Loading SBERT model: sentence-transformers/all-MiniLM-L6-v2


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
# pipelines = {
#     'SVM': Pipeline([
#         ('sbert', preprocessor),
#         ('scaler', StandardScaler()),
#         ('classifier', SVC(random_state=42, probability=True))  # CPU only
#     ]),
    
#     'Logistic Regression': Pipeline([
#         ('sbert', preprocessor),
#         ('scaler', StandardScaler()),
#         ('classifier', LogisticRegression(random_state=42, max_iter=1000))  # CPU only
#     ]),
    
#     'Random Forest': Pipeline([
#         ('sbert', preprocessor),
#         ('classifier', RandomForestClassifier(random_state=42))  # CPU only
#     ]),

#     'LGBMClassifier': Pipeline([
#         ('sbert', preprocessor),
#         ('classifier', LGBMClassifier(random_state=42, device='gpu'))  # GPU enabled
#     ]),

#     'KNeighborsClassifier': Pipeline([
#         ('sbert', preprocessor),
#         ('classifier', KNeighborsClassifier())  # CPU only
#     ]),

#     'XGBClassifier': Pipeline([
#         ('sbert', preprocessor),
#         ('classifier', XGBClassifier(random_state=42, tree_method='gpu_hist', predictor='gpu_predictor'))  # GPU enabled
#     ]),

#     'CatBoostClassifier': Pipeline([
#         ('sbert', preprocessor),
#         ('classifier', CatBoostClassifier(random_state=42, task_type='GPU', verbose=0))  # GPU enabled
#     ])
# }

In [10]:
# for name, pipe in pipelines.items():
#     scores = cross_val_score(pipe, X, y, cv=5, scoring='f1')
#     print(f"{name} Mean F1: {scores.mean():.4f}")

In [11]:
pipe=Pipeline([
        ('sbert', preprocessor),
        ('classifier', CatBoostClassifier(random_state=42, task_type='GPU', verbose=0))  # GPU enabled
    ])

In [12]:
best_estimator=pipe.fit(X,y)

SVM Mean F1: 0.7423
Logistic Regression Mean F1: 0.7298
Random Forest Mean F1: 0.7158
LGBMClassifier Mean F1: 0.7324
KNeighborsClassifier Mean F1: 0.7160
XGBClassifier Mean F1: 0.7286
CatBoostClassifier Mean F1: 0.7425

+++++++++++++++++++++++++++++TEST+++++++++++++++++++++++++

In [13]:
df = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [14]:
X=df.drop(columns=["id"])

In [15]:
predictions = best_estimator.predict(X)

In [16]:
submission = pd.DataFrame({
    "id": df["id"],
    "target": predictions
})

submission.to_csv("submission.csv", index=False)
print("Saved submission.csv for Kaggle")

Saved submission.csv for Kaggle
